## Imports

In [4]:
import flopy
import numpy as np
import netCDF4 as nc
import modflowapi  # needs numpy, pandas, and xmipy
import numpy as np
import pandas as pd
from datetime import datetime
import os
import time

## Create VIC Baseflow netCDF

In [7]:
# create synthetic netcdf to mimic vic baseflow
nrows, ncols = 912, 328  # model grid size
ntimes = 10  # test with 10 stress periods
start_date = datetime(1990, 1, 1)  # arbitrary start date

# generate random baseflow (mm/day, 0.1 to 5)
np.random.seed(7910)
baseflow = np.random.uniform(0.1, 5.0, size=(ntimes, nrows, ncols))

# create netcdf file
ds = nc.Dataset("vic_baseflow.nc", "w", format="NETCDF4")
ds.createDimension("time", ntimes)
ds.createDimension("row", nrows)
ds.createDimension("col", ncols)

# time variable
time_var = ds.createVariable("time", "f8", ("time",))
time_var.units = f"days since {start_date.strftime('%Y-%m-%d')}"
time_var[:] = np.arange(ntimes)

# baseflow variable
baseflow_var = ds.createVariable("baseflow", "f4", ("time", "row", "col"))
baseflow_var.units = "mm/day"
baseflow_var[:] = baseflow

ds.close()
print("created vic_baseflow.nc")

created vic_baseflow.nc


## load sim

In [8]:
# load existing mf6 simulation
workspace = "../../MF6/rgtihm/model/"
sim_name = "rgtihm"
sim = flopy.mf6.MFSimulation.load(
    sim_name=sim_name, version="6.7.0.dev1", exe_name="mf6", sim_ws=workspace
)
gwf = sim.get_model()
uzf = gwf.uzf
print("simulation loaded successfully")

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package oc...
    loading package ic...
    loading package ghb...
    loading package npf...
    loading package sto...
    loading package hfb...
    loading package uzf...
  loading solution package rgtihm...
simulation loaded successfully


## init params

In [9]:
# set model parameters
nrow, ncol = 912, 328  # grid dimensions
nuzfcells = 17879  # active uzf cells in layer 1
nper = 10  # test with 10 stress periods
mm_to_ft = 1 / 304.8  # convert mm to ft for finf

# get active uzf cells from idomain
idomain = gwf.modelgrid.idomain
active_cells = np.argwhere(idomain[0] == 1)  # shape: (17879, 2)
print(f"active uzf cells: {len(active_cells)}")

active uzf cells: 17879


## Load netCDF

In [6]:
# load netcdf baseflow data
nc_file = "vic_baseflow.nc"
ds = nc.Dataset(nc_file)
baseflow = ds.variables["baseflow"][:]  # shape: (10, 912, 328), mm/day
print(f"loaded netcdf with shape: {baseflow.shape}")

loaded netcdf with shape: (10, 912, 328)


## init api

In [10]:
# initialize mf6 api
try:
    mf6 = modflowapi.ModflowApi("/home/abd/usr/local/src/modflow6/bin/libmf6.so", working_directory=workspace)  # use modflowapi with shared library
    mf6.initialize()
    print("mf6 api initialized")
except OSError as e:
    print(f"error: failed to load mf6 library - {e}")
    print("ensure /home/abd/usr/local/src/modflow6/bin/libmf6.so exists and is accessible")
    raise

mf6 api initialized


## setup log

In [11]:
# create log file for finf and uzf-gwd
log_file = "coupling_log.csv"
with open(log_file, "w") as f:
    f.write("stress_period,iuzno,finf_ft_per_day,uzf_gwd_ft_per_day\n")
print(f"log file created: {log_file}")

log file created: coupling_log.csv


## run coupling 

# run coupling loop for each stress period
for t in range(nper):
    print(f"stress period {t+1}/{nper}: starting baseflow reading")
    # read baseflow and convert to ft/day
    finf_grid = baseflow[t] * mm_to_ft  # shape: (912, 328)
    print(f"stress period {t+1}/{nper}: read baseflow, finf range (ft/day): "
          f"min={finf_grid.min():.6f}, max={finf_grid.max():.6f}, mean={finf_grid.mean():.6f}")

    print(f"stress period {t+1}/{nper}: starting uzf period data preparation")
    # prepare uzf period data
    period_data = []
    for idx, (i, j) in enumerate(active_cells[:10]):  # limit to first 10 cells
        iuzno = idx  # 0 to 9
        finf = finf_grid[i, j]
        period_data.append(
            [iuzno, finf, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  # finf, pet, extdp, extwc, ha, hroot, rootact
        )
    print(f"stress period {t+1}/{nper}: prepared uzf period data for {len(period_data)} cells")

    print(f"stress period {t+1}/{nper}: starting uzf period data update")
    # update uzf period data
    uzf.perioddata = {t: period_data}
    uzf.write()
    print(f"stress period {t+1}/{nper}: updated uzf period data")

    print(f"stress period {t+1}/{nper}: starting mf6 stress period run")
    # run one stress period
    mf6.prepare_time_step(t)
    mf6.do_time_step()
    mf6.finalize_time_step()
    print(f"stress period {t+1}/{nper}: ran mf6 stress period")

    print(f"stress period {t+1}/{nper}: starting uzf-gwd extraction")
    # extract uzf-gwd from observation file
    uzf_gwd = np.zeros(10)  # size matches 10 cells
    try:
        import pandas as pd
        obs_data = pd.read_csv("../../MF6/rgtihm/model/rgtihm.obs.gwf.csv")
        # assume stress period t corresponds to row t (adjust if needed _ not sure about the indexing yet)
        if t < len(obs_data):
            gwd_data = obs_data.iloc[t]["GWD"]
            # assume GWD is a single value or array; adjust for first 10 cells
            uzf_gwd[:] = gwd_data if isinstance(gwd_data, float) else gwd_data[:10]
        else:
            print(f"stress period {t+1}/{nper}: warning: no obs data for period, using zeros")
    except Exception as e:
        print(f"stress period {t+1}/{nper}: error reading obs file - {e}")
        print(f"stress period {t+1}/{nper}: using zeros for uzf-gwd")
    print(f"stress period {t+1}/{nper}: extracted uzf-gwd, range (ft/day): "
          f"min={uzf_gwd.min():.6f}, max={uzf_gwd.max():.6f}, mean={uzf_gwd.mean():.6f}")

    print(f"stress period {t+1}/{nper}: starting logging results")
    # log results
    with open(log_file, "a") as f:
        for idx, (i, j) in enumerate(active_cells[:10]):  # limit to first 10 cells
            iuzno = idx
            f.write(f"{t},{iuzno},{period_data[idx][1]:.6f},{uzf_gwd[iuzno]:.6f}\n")
    print(f"stress period {t+1}/{nper}: logged results to {log_file}")

    print(f"completed stress period {t+1}/{nper}")

## complete coupled cycle

In [ ]:
# dummy VIC function: generates baseflow, writes to CSV in current dir, takes 10 seconds
def run_dummy_vic(prev_gwd_mm_day=0.0, stress_period=0):
    print(f"stress period {stress_period+1}: running dummy VIC (10 seconds)...")
    time.sleep(10)
    # generate random baseflow (0.1–5 mm/day), influenced by prev_gwd
    baseflow = np.random.uniform(0.1, 5.0) + (prev_gwd_mm_day * 0.1)  # slight GWD influence
    baseflow = min(max(baseflow, 0.1), 5.0)  # clamp to 0.1–5 mm/day
    print(f"stress period {stress_period+1}: dummy VIC generated baseflow: {baseflow:.6f} mm/day")
    # write to CSV in current working directory
    csv_path = "./vic_baseflow.csv"
    df = pd.DataFrame({"stress_period": [stress_period], "baseflow_mm_day": [baseflow]})
    # append to CSV (create if first period)
    mode = "a" if stress_period > 0 else "w"
    df.to_csv(csv_path, mode=mode, index=False, header=(stress_period == 0))
    print(f"stress period {stress_period+1}: dummy VIC wrote baseflow to {csv_path}")
    return baseflow

# run coupling loop for each stress period
prev_gwd_mm_day = 0.0  # initial GWD for VIC
for t in range(nper):
    print(f"stress period {t+1}/{nper}: starting dummy VIC")
    # run dummy VIC to get baseflow and write CSV
    baseflow_val = run_dummy_vic(prev_gwd_mm_day, t)  # mm/day

    print(f"stress period {t+1}/{nper}: starting baseflow reading")
    # read baseflow from CSV
    try:
        baseflow_df = pd.read_csv("./vic_baseflow.csv")
        baseflow_row = baseflow_df[baseflow_df["stress_period"] == t]
        if not baseflow_row.empty:
            baseflow_val = baseflow_row["baseflow_mm_day"].iloc[0]
        else:
            print(f"stress period {t+1}/{nper}: warning: no baseflow data for period, using dummy value")
            baseflow_val = np.random.uniform(0.1, 5.0)
    except Exception as e:
        print(f"stress period {t+1}/{nper}: error reading baseflow CSV - {e}")
        print(f"stress period {t+1}/{nper}: using dummy baseflow")
        baseflow_val = np.random.uniform(0.1, 5.0)
    finf_val = baseflow_val * mm_to_ft
    print(f"stress period {t+1}/{nper}: read baseflow, finf: {finf_val:.6f} ft/day")

    print(f"stress period {t+1}/{nper}: starting uzf period data preparation")
    # prepare uzf period data with single finf value for 10 cells
    period_data = []
    for idx, (i, j) in enumerate(active_cells[:10]):  # limit to first 10 cells
        iuzno = idx  # 0 to 9
        period_data.append(
            [iuzno, finf_val, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  # finf, pet, extdp, extwc, ha, hroot, rootact
        )
    print(f"stress period {t+1}/{nper}: prepared uzf period data for {len(period_data)} cells")

    print(f"stress period {t+1}/{nper}: starting uzf period data update")
    # update uzf period data
    uzf.perioddata = {t: period_data}
    uzf.write()
    print(f"stress period {t+1}/{nper}: updated uzf period data")

    print(f"stress period {t+1}/{nper}: starting mf6 stress period run")
    # run one stress period with 2 timesteps using ModflowApi
    for ts in range(2):  # 2 timesteps
        print(f"stress period {t+1}/{nper}: running timestep {ts+1}/2")
        mf6.update()  # run one timestep
    print(f"stress period {t+1}/{nper}: ran mf6 stress period (2 timesteps)")

    print(f"stress period {t+1}/{nper}: starting uzf-gwd extraction")
    # extract uzf-gwd from observation file
    uzf_gwd = np.zeros(10)  # size matches 10 cells
    try:
        obs_data = pd.read_csv("../../MF6/rgtihm/model/rgtihm.obs.gwf.csv")
        if t < len(obs_data):
            gwd_data = obs_data.iloc[t]["GWD"]
            # assume GWD is a single value; apply to all 10 cells
            uzf_gwd[:] = gwd_data if isinstance(gwd_data, float) else gwd_data[:10]
            if np.all(uzf_gwd == 0):
                print(f"stress period {t+1}/{nper}: uzf-gwd is zero, using synthetic GWD")
                uzf_gwd[:] = finf_val * np.random.uniform(0.1, 0.5)  # 0.1–0.5 × finf
        else:
            print(f"stress period {t+1}/{nper}: warning: no obs data for period, using synthetic GWD")
            uzf_gwd[:] = finf_val * np.random.uniform(0.1, 0.5)  # 0.1–0.5 × finf
    except Exception as e:
        print(f"stress period {t+1}/{nper}: error reading obs file - {e}")
        print(f"stress period {t+1}/{nper}: using synthetic GWD")
        uzf_gwd[:] = finf_val * np.random.uniform(0.1, 0.5)  # 0.1–0.5 × finf
    print(f"stress period {t+1}/{nper}: extracted uzf-gwd, range (ft/day): "
          f"min={uzf_gwd.min():.6f}, max={uzf_gwd.max():.6f}, mean={uzf_gwd.mean():.6f}")

    # convert uzf-gwd to mm/day for VIC
    prev_gwd_mm_day = uzf_gwd.mean() * 1000 / 3.28084  # average GWD to single value
    print(f"stress period {t+1}/{nper}: passing GWD to VIC: {prev_gwd_mm_day:.6f} mm/day")

    print(f"stress period {t+1}/{nper}: starting logging results")
    # log results
    with open(log_file, "a") as f:
        for idx, (i, j) in enumerate(active_cells[:10]):  # limit to first 10 cells
            iuzno = idx
            f.write(f"{t},{iuzno},{period_data[idx][1]:.6f},{uzf_gwd[iuzno]:.6f}\n")
    print(f"stress period {t+1}/{nper}: logged results to {log_file}")

    print(f"completed stress period {t+1}/{nper}")

stress period 1/10: starting dummy VIC
stress period 1: running dummy VIC (10 seconds)...
stress period 1: dummy VIC generated baseflow: 4.574815 mm/day
stress period 1: dummy VIC wrote baseflow to ./vic_baseflow.csv
stress period 1/10: starting baseflow reading
stress period 1/10: read baseflow, finf: 0.015009 ft/day
stress period 1/10: starting uzf period data preparation
stress period 1/10: prepared uzf period data for 10 cells
stress period 1/10: starting uzf period data update
stress period 1/10: updated uzf period data
stress period 1/10: starting mf6 stress period run
stress period 1/10: running timestep 1/2
stress period 1/10: running timestep 2/2
stress period 1/10: ran mf6 stress period (2 timesteps)
stress period 1/10: starting uzf-gwd extraction
stress period 1/10: uzf-gwd is zero, using synthetic GWD
stress period 1/10: extracted uzf-gwd, range (ft/day): min=0.007244, max=0.007244, mean=0.007244
stress period 1/10: passing GWD to VIC: 2.208051 mm/day
stress period 1/10: st

## callback trail

In [ ]:
import numpy as np
import pandas as pd
import modflowapi
from modflowapi import Callbacks, run_simulation
import time
import os
import logging
from datetime import datetime
import psutil

# Set up logging to a file for error debugging
logging.basicConfig(
    filename="coupling_error.log",
    level=logging.ERROR,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# Class to handle VIC-MF6 coupling
class VicCouplingMonitor:
    def __init__(self, nper, active_cells, log_file, mm_to_ft=0.00328084):
        self.nper = nper
        self.active_cells = active_cells  # Shape: (17879, 2) for row, col
        self.log_file = log_file
        self.mm_to_ft = mm_to_ft
        self.prev_gwd_mm_day = 0.0  # Initial GWD for VIC
        self.current_period = 0
        self.baseflow_val = 0.0
        self.timing = {}  # Store timing for performance analysis
        self.memory_usage = []  # Track memory usage

    def log_memory_usage(self):
        """Log current memory usage to track potential memory issues."""
        try:
            process = psutil.Process()
            mem_info = process.memory_info()
            mem_mb = mem_info.rss / 1024 / 1024  # Convert to MB
            self.memory_usage.append((self.current_period, mem_mb))
            print(f"Stress period {self.current_period+1}: Memory usage: {mem_mb:.2f} MB")
        except Exception as e:
            logging.error(f"Error logging memory usage: {e}")
            print(f"Error logging memory usage: {e}")

    def run_dummy_vic(self, stress_period):
        """Simulate dummy VIC: generate baseflow, write to CSV, delay 10 seconds."""
        print(f"Stress period {stress_period+1}: Running dummy VIC (10 seconds)...")
        start_time = time.time()
        try:
            time.sleep(10)
            # Generate random baseflow (0.1–5 mm/day), influenced by prev_gwd
            baseflow = np.random.uniform(0.1, 5.0) + (self.prev_gwd_mm_day * 0.1)
            baseflow = min(max(baseflow, 0.1), 5.0)
            print(f"Stress period {stress_period+1}: Dummy VIC generated baseflow: {baseflow:.6f} mm/day")
            # Write to CSV in current working directory
            csv_path = "./vic_baseflow.csv"
            df = pd.DataFrame({"stress_period": [stress_period], "baseflow_mm_day": [baseflow]})
            mode = "a" if stress_period > 0 else "w"
            df.to_csv(csv_path, mode=mode, index=False, header=(stress_period == 0))
            print(f"Stress period {stress_period+1}: Dummy VIC wrote baseflow to {csv_path}")
        except Exception as e:
            logging.error(f"Error in run_dummy_vic for stress period {stress_period+1}: {e}")
            print(f"Stress period {stress_period+1}: Error writing vic_baseflow.csv - {e}")
            raise
        self.timing[f"vic_sp{stress_period}"] = time.time() - start_time
        return baseflow

    def callback(self, sim, callback_step):
        """Callback function for MODFLOW-API simulation."""
        try:
            self.log_memory_usage()  # Log memory usage at each callback

            if callback_step == Callbacks.initialize:
                print("Initializing MODFLOW-API simulation")
                self.mf6 = modflowapi.ModflowApi(dll, working_directory=workspace)
                try:
                    self.mf6.initialize()
                except Exception as e:
                    logging.error(f"Error initializing ModflowApi: {e}")
                    print(f"Error initializing ModflowApi: {e}")
                    raise
                print("ModflowApi initialized")

            if callback_step == Callbacks.stress_period_start:
                t = self.current_period
                print(f"Stress period {t+1}/{self.nper}: Starting coupling")

                # Run dummy VIC and write CSV
                start_time = time.time()
                try:
                    self.baseflow_val = self.run_dummy_vic(t)
                except Exception as e:
                    logging.error(f"Error running dummy VIC for stress period {t+1}: {e}")
                    print(f"Stress period {t+1}: Error running dummy VIC - {e}")
                    raise
                self.timing[f"vic_total_sp{t}"] = time.time() - start_time

                # Read baseflow from CSV
                start_time = time.time()
                try:
                    baseflow_df = pd.read_csv("./vic_baseflow.csv")
                    baseflow_row = baseflow_df[baseflow_df["stress_period"] == t]
                    if not baseflow_row.empty:
                        self.baseflow_val = baseflow_row["baseflow_mm_day"].iloc[0]
                    else:
                        print(f"Stress period {t+1}/{self.nper}: Warning: No baseflow data, using dummy value")
                        self.baseflow_val = np.random.uniform(0.1, 5.0)
                except Exception as e:
                    logging.error(f"Error reading baseflow CSV for stress period {t+1}: {e}")
                    print(f"Stress period {t+1}/{self.nper}: Error reading baseflow CSV - {e}")
                    self.baseflow_val = np.random.uniform(0.1, 5.0)
                finf_val = self.baseflow_val * self.mm_to_ft
                print(f"Stress period {t+1}/{self.nper}: Read baseflow, finf: {finf_val:.6f} ft/day")
                self.timing[f"baseflow_read_sp{t}"] = time.time() - start_time

                # Update UZF finf using ModflowApi
                start_time = time.time()
                try:
                    uzf_finf = self.mf6.get_value("UZF_0/INFILTRATION")
                    for idx in range(10):  # First 10 cells
                        uzf_finf[idx] = finf_val  # Update finf directly
                    self.mf6.set_value("UZF_0/INFILTRATION", uzf_finf)
                    print(f"Stress period {t+1}/{self.nper}: Updated UZF finf for 10 cells")
                except (ValueError, MemoryError, OSError) as e:
                    logging.error(f"Error updating UZF finf for stress period {t+1}: {e}")
                    print(f"Stress period {t+1}/{self.nper}: Error updating UZF finf - {e}")
                    raise
                self.timing[f"uzf_update_sp{t}"] = time.time() - start_time

            if callback_step == Callbacks.stress_period_end:
                t = self.current_period
                print(f"Stress period {t+1}/{self.nper}: Extracting uzf-gwd")

                # Extract uzf-gwd from observation file
                start_time = time.time()
                uzf_gwd = np.zeros(10)
                try:
                    obs_data = pd.read_csv("../../MF6/rgtihm/model/rgtihm.obs.gwf.csv")
                    if t < len(obs_data):
                        gwd_data = obs_data.iloc[t]["GWD"]
                        uzf_gwd[:] = gwd_data if isinstance(gwd_data, float) else gwd_data[:10]
                        if np.all(uzf_gwd == 0):
                            print(f"Stress period {t+1}/{self.nper}: uzf-gwd is zero, using synthetic GWD")
                            uzf_gwd[:] = self.baseflow_val * self.mm_to_ft * np.random.uniform(0.1, 0.5)
                    else:
                        print(f"Stress period {t+1}/{self.nper}: Warning: No obs data, using synthetic GWD")
                        uzf_gwd[:] = self.baseflow_val * self.mm_to_ft * np.random.uniform(0.1, 0.5)
                except Exception as e:
                    logging.error(f"Error reading obs file for stress period {t+1}: {e}")
                    print(f"Stress period {t+1}/{self.nper}: Error reading obs file - {e}")
                    uzf_gwd[:] = self.baseflow_val * self.mm_to_ft * np.random.uniform(0.1, 0.5)
                print(f"Stress period {t+1}/{self.nper}: Extracted uzf-gwd, range (ft/day): "
                      f"min={uzf_gwd.min():.6f}, max={uzf_gwd.max():.6f}, mean={uzf_gwd.mean():.6f}")
                self.timing[f"gwd_extract_sp{t}"] = time.time() - start_time

                # Convert uzf-gwd to mm/day for VIC
                self.prev_gwd_mm_day = uzf_gwd.mean() * 1000 / 3.28084
                print(f"Stress period {t+1}/{self.nper}: Passing GWD to VIC: {self.prev_gwd_mm_day:.6f} mm/day")

                # Log results
                start_time = time.time()
                try:
                    with open(self.log_file, "a") as f:
                        for idx, (i, j) in enumerate(self.active_cells[:10]):
                            iuzno = idx
                            f.write(f"{t},{iuzno},{self.baseflow_val * self.mm_to_ft:.6f},{uzf_gwd[iuzno]:.6f}\n")
                    print(f"Stress period {t+1}/{self.nper}: Logged results to {self.log_file}")
                except Exception as e:
                    logging.error(f"Error writing to log file for stress period {t+1}: {e}")
                    print(f"Stress period {t+1}/{self.nper}: Error writing to log file - {e}")
                    raise
                self.timing[f"log_sp{t}"] = time.time() - start_time

                self.current_period += 1
                print(f"Completed stress period {t+1}/{self.nper}")

            if callback_step == Callbacks.finalize:
                print("Finalizing simulation")
                # Print timing summary
                print("Timing summary (seconds):")
                for key, value in self.timing.items():
                    print(f"{key}: {value:.2f}")
                # Print memory usage summary
                print("Memory usage summary (MB):")
                for period, mem_mb in self.memory_usage:
                    print(f"Stress period {period+1}: {mem_mb:.2f} MB")
                try:
                    self.mf6.finalize()
                except Exception as e:
                    logging.error(f"Error finalizing simulation: {e}")
                    print(f"Error finalizing simulation: {e}")

        except Exception as e:
            logging.error(f"Callback error in stress period {self.current_period+1}: {e}")
            print(f"Callback error in stress period {self.current_period+1}: {e}")
            raise

# Assuming simulation is pre-loaded as `sim` and `gwf` is available
# Derive active_cells from pre-loaded gwf
active_cells = np.argwhere(gwf.modelgrid.idomain[0] == 1)
nper = 10
log_file = "coupling_log.csv"
dll = "/home/abd/usr/local/src/modflow6/bin/libmf6.so"
workspace = "../../MF6/rgtihm/model/"

# Initialize and run simulation with callback
monitor = VicCouplingMonitor(nper=nper, active_cells=active_cells, log_file=log_file)
try:
    run_simulation(dll, workspace, monitor.callback, verbose=True)
except (ValueError, MemoryError, OSError) as e:
    logging.error(f"Simulation failed: {e}")
    print(f"Simulation failed: {e}")

MODFLOW-6 API Version 6.7.0.dev1
Initializing MODFLOW-6 simulation


## finalize

# finalize simulation and close netcdf
mf6.finalize()
ds.close()
print("simulation complete. results in coupling_log.csv")